This script download CHIRPS data and calculates the monthly thresholds for CHIRPS precipitation data. It can be adapted to any other monthly precipitation dataset.<br>
Script by Danielle Touma (detouma@ucar.edu).

## Set up

### Import libraries/packages

In [1]:
import numpy as np
import scipy.stats as st
import netCDF4
from netCDF4 import Dataset 
import calendar as cd
from calendar import monthrange
import glob
import sys
import urllib

### Specify directories for file reading and writing

In [2]:
dir_pxx = "/glade/scratch/detouma/adaptation/CHIRPS_monthly_pr/"
dir_pr = "/glade/scratch/detouma/adaptation/CHIRPS_monthly_pr/"

### Download CHIRPS dataset

In [3]:
# can take ~10 minutes
chirps_out = dir_pr+'chirps_monthly_dataset.nc'
chirps_url = 'https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_monthly/netcdf/chirps-v2.0.monthly.nc'
urllib.request.urlretrieve(chirps_url,chirps_out)

('/glade/scratch/detouma/adaptation/CHIRPS_monthly_pr/chirps_monthly_dataset.nc',
 <http.client.HTTPMessage at 0x2b097b9adeb0>)

### Specify percentile thresholds of interest

In [4]:
pxx_list = [2,5,10,20,90,95,99]
npxx = len(pxx_list)

### Variable initialization

In [5]:
# Variable initialization
year0 = 1981
year1 = 2021
years = np.arange(year0,(year1+1),1)
nyears = len(years)
nmonths = nyears*12

### Read in monthly precipitation file and subset

In [6]:
file_in = chirps_out
nc_in = Dataset(file_in,'r')
ntime = nc_in.dimensions['time'].size
lat0 = nc_in.variables['latitude'][:]
lon0 = nc_in.variables['longitude'][:]
lat_units = nc_in.variables['latitude'].units
lon_units = nc_in.variables['longitude'].units

lat_inds = np.where((lat0>=-40)&(lat0<=40))[0]
lon_inds = np.where((lon0>=-20)&(lon0<=60))[0]
lat = lat0[lat_inds]
lon = lon0[lon_inds]
nlat = len(lat)
nlon = len(lon)

precip_month0 = nc_in.variables['precip'][0:nmonths,lat_inds,lon_inds] 
precip_month = np.reshape(precip_month0,newshape=(int(nmonths/12),12,nlat,nlon))

## Processing

### Calculate percentile per month

In [7]:
pxx_month = np.percentile(precip_month,pxx_list,axis=0)

/glade/u/apps/opt/conda/envs/npl/lib/python3.8/site-packages/numpy/lib/function_base.py:4691: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


In [8]:
#mask missing values
pxx_month = np.ma.masked_array(pxx_month,mask=(pxx_month<0))

## Output

### Write percentiles file

In [9]:
file_out = dir_pxx+'CHIRPS_'+str(year0)+'-'+str(year1)+'_pr_monthly_percentiles_africa.nc'
nc_out = Dataset(file_out,'w',format='NETCDF4')
nc_out.description = 'percentiles of CHIRPS monthly precipitation ' + str(year0) + ' to ' + str(year1)
nc_out.createDimension('month',12)
nc_out.createDimension('pxx',npxx)
nc_out.createDimension('lat',nlat)
nc_out.createDimension('lon',nlon)
nc_out.createVariable('pxx',lat.dtype,('pxx',))
nc_out.createVariable('lat',lat.dtype,('lat',))
nc_out.createVariable('lon',lon.dtype,('lon',))
nc_out.createVariable('pr',pxx_month.dtype,('pxx','month','lat','lon'))
nc_out.variables['pxx'][:] = np.array(pxx_list)
nc_out.variables['lat'][:] = lat
nc_out.variables['lon'][:] = lon
nc_out.variables['pr'][:] = pxx_month
nc_out.variables['lat'].units = lat_units 
nc_out.variables['lon'].units = lon_units 
nc_out.variables['pr'].units = 'mm/month' 
nc_out.close()